In [ ]:
# !pip install elasticsearch==8.13.0
#!pip install  transformers torch sentence-transformers numpy 

In [ ]:
from sentence_transformers import SentenceTransformer 
import numpy as np

Hugging_face_token=''

model=SentenceTransformer(
    "intfloat/multilingual-e5-base",
    use_auth_token=Hugging_face_token
)

def chunk_text(text,maxLength=400):
    words=text.split() 
    chunks=[]
    current_chunk=[]
    current_length=0


    for word in words:
        current_length+=len(word)+1 
        if current_length>maxLength:
            chunks.append(' '.join(current_chunk))
            current_chunk=[word]
            current_length=len(word)+1 
        else:
            current_chunk.append(word)
        if current_chunk:
            chunks.append(' '.join(current_chunk))
        return chunks 

In [ ]:
#read the md file data 
with open('parsed-doc/Sukanya Samriddhi Account Scheme 2019 English (1)-with-image-links.md', 'r', encoding='utf-8') as file:
     text=file.read() 

chunks=chunk_text(text)
embeddings=[]

for chunk in chunks:
    embedding=model.encode(f"passage:{chunk}",noramlize_embeddings=True)
    embedding.append(embedding.tolist())


np.save('embedding.npy',embedding)

print(f"number of chunks {len(chunks)}")
print(f"sample embedding dimmension :{len(embedding[0])}")


In [ ]:
from elasticsearch import Elasticsearch 

es=Elasticsearch(['http://localhost:9200'])

mappings={
    "properties":{
        "text":{"type":"text"},
        "embedding":{
            "type":"dense_vector",
            "dims":768,
            "index":True,
            "similarity":"cosine"
        },
        "language":{"type":"keyword"}
    }
}

es.indices.create(index='markdown_vectors',body={"mappings":mappings},ignore=400)

print("Index 'markdown_vectors' created with 768-dimensional dense_vector field.") 
